In [1]:
import tensorflow as tf
import keras
print(f'keras version = {keras.__version__}')
print(f'Tensorflow version = {tf.__version__}')

keras version = 3.4.1
Tensorflow version = 2.17.0


In [2]:
import numpy as np 
import pandas as pd
import sklearn

In [3]:
from sklearn import preprocessing

In [4]:
keras.Sequential

keras.src.models.sequential.Sequential

In [5]:
keras.__version__

'3.4.1'

In [6]:
keras.layers.Dense

keras.src.layers.core.dense.Dense

In [7]:
df = pd.read_csv('../dataset/BankChurners.csv')

In [8]:
df.columns = [i.lower().replace(' ','_') for i in df.columns]

In [9]:
df.head()

,attrition_flag,customer_age,gender,dependent_count,education_level,marital_status,income_category,card_category,months_on_book,total_relationship_count,...,avg_open_to_buy,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio,unnamed:_20,unnamed:_21,unnamed:_22,unnamed:_23
0,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,...,11914.0,1.335,1144,42,1.625,0.061,NaN,NaN,NaN,NaN
1,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,...,7392.0,1.541,1291,33,3.714,0.105,NaN,NaN,NaN,NaN
2,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,...,3418.0,2.594,1887,20,2.333,0.000,NaN,NaN,NaN,NaN
3,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,...,796.0,1.405,1171,20,2.333,0.760,NaN,NaN,NaN,NaN
4,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,...,4716.0,2.175,816,28,2.500,0.000,NaN,NaN,NaN,NaN


In [10]:
dict.fromkeys(df.attrition_flag.value_counts().index)

{'Existing Customer': None, 'Attrited Customer': None}

In [11]:
target_mappings = {'Existing Customer': 0, 'Attrited Customer': 1}

In [12]:
df['target'] = df['attrition_flag'].map(target_mappings)

In [13]:
drop_columns = ['attrition_flag','gender','education_level','marital_status','income_category','card_category']
drop_columns_part1 = [i for i in df.columns if 'unnamed' in i]
drop_cols_final= drop_columns + drop_columns_part1 
print(drop_cols_final)

['attrition_flag', 'gender', 'education_level', 'marital_status', 'income_category', 'card_category', 'unnamed:_20', 'unnamed:_21', 'unnamed:_22', 'unnamed:_23']


In [14]:
df.drop(drop_cols_final,axis=1,inplace = True)

In [15]:
df.shape

(10127, 15)

In [16]:
# df.head()

In [17]:
def scale_data(data,method='standard',feature_range=(0,1)):
   if len(data.shape) ==1:
       data = data.reshape(-1,1)
   if method.lower()=='standard':
       scaler = preprocessing.StandardScaler()
   elif method.lower()=='minmax':
       scaler = preprocessing.MinMaxScaler(feature_range=feature_range)
   else:
       raise ValueError('Method must be either "standard" or "minmax"')
   scaled_data = scaler.fit_transform(data)
   return scaled_data,scaler

def inverse_scale_data(scaled_data,scaler):
   if len(scaled_data.shape)==1:
       scaled_data = scaled_data.reshape(-1,1)
   original_data = scaler.inverse_transform(scaled_data)
   return original_data

In [18]:
x = df.drop('target',axis=1)
y = df['target']
features = x.columns.tolist()

In [19]:
x_scaled,scaler = scale_data(x)

In [20]:
# pd.DataFrame(inverse_scale_data(x_scaled,scaler),columns = features).head()

In [21]:
x.shape,y.shape

((10127, 14), (10127,))

In [22]:
indexes_of_ones = np.where(y == 1)[0]
indexes_of_ones

array([   21,    39,    51, ..., 10124, 10125, 10126])

In [23]:
len(indexes_of_ones)

1627

In [24]:
y.value_counts()

target
0    8500
1    1627
Name: count, dtype: int64

In [25]:
records_with_ones = x_scaled[indexes_of_ones]
records_with_ones.shape

(1627, 14)

In [26]:
np.random.seed(101)
# Define the generator network
def build_generator(latent_dim, output_dim):
    model = keras.Sequential()
    model.add(keras.layers.Dense(64, input_shape=(latent_dim,)))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(8,activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(output_dim, activation='sigmoid'))
    return model
# Dimensionality of the input noise for the generator
latent_dim = 128
len_features = len(features)
generator = build_generator(latent_dim, len_features)

/Users/thomas/Documents/GitHub/Deep-Learning/.conda/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
np.random.seed(101)
# Define the discriminator network
def build_discriminator(input_dim):
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape = (input_dim,)))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(8, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    return model

discriminator = build_discriminator(len_features)
print(discriminator.summary())

/Users/thomas/Documents/GitHub/Deep-Learning/.conda/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 8)              │           264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,425 (21.19 KB)

 Trainable params: 5,425 (21.19 KB)

 Non-trainable params: 0 (0.00 B)

None


In [28]:
def precision(y_true, y_pred):
    """
    Computes precision using TensorFlow.

    Args:
        y_true: Ground truth (binary labels, tensor).
        y_pred: Predicted values (probabilities or binary predictions, tensor).

    Returns:
        Precision value (tensor scalar).
    """
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    
    epsilon = tf.keras.backend.epsilon()  # Small constant for numerical stability
    y_pred_rounded = tf.round(tf.clip_by_value(y_pred, 0, 1))
    
    true_positives = tf.reduce_sum(tf.round(y_true * y_pred_rounded))
    predicted_positives = tf.reduce_sum(y_pred_rounded)
    
    precision_value = true_positives / (predicted_positives + epsilon)
    return precision_value


def recall(y_true, y_pred):
    """
    Computes recall using TensorFlow.

    Args:
        y_true: Ground truth (binary labels, tensor).
        y_pred: Predicted values (probabilities or binary predictions, tensor).

    Returns:
        Recall value (tensor scalar).
    """
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    
    epsilon = tf.keras.backend.epsilon()  # Small constant for numerical stability
    y_pred_rounded = tf.round(tf.clip_by_value(y_pred, 0, 1))
    
    true_positives = tf.reduce_sum(tf.round(y_true * y_pred_rounded))
    possible_positives = tf.reduce_sum(tf.round(y_true))
    
    recall_value = true_positives / (possible_positives + epsilon)
    return recall_value


discriminator.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001,beta_1=0.5),
    loss='binary_crossentropy',  
    metrics=[precision,recall]
)

def generator_loss_log_d(y_true, y_pred):
    """
    Generator loss for GANs implemented using TensorFlow operations.
    Encourages y_pred values to approach 1.

    Args:
        y_true: Ground truth labels (tensor).
        y_pred: Predicted probabilities (tensor).

    Returns:
        Scalar loss value (tensor).
    """
    epsilon = tf.keras.backend.epsilon()  # Small constant to prevent log(0)
    y_pred = tf.clip_by_value(y_pred, epsilon, 1.0)  # Ensure y_pred is within [epsilon, 1]
    return -tf.reduce_mean(tf.math.log(y_pred))


# Build and compile the GANs upper optimization loop combining generator and discriminator
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5),
        loss=generator_loss_log_d
    )
    return model

gan = build_gan(generator, discriminator)
print(gan.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 14)             │        12,838 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 1)              │         5,425 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,263 (71.34 KB)

 Trainable params: 12,838 (50.15 KB)

 Non-trainable params: 5,425 (21.19 KB)

None


In [29]:
# Set hyperparameters
import os 
import logging
epochs = 100
batch_size = 8
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# Redirect TensorFlow logs to a file
log_file = open("tensorflow_logs.txt", "w")
tf.get_logger().addHandler(logging.StreamHandler(log_file))

In [68]:
# Training loop for the GANs
for epoch in range(epochs):
    discriminator.trainable = True
    generator.trainable = False

    # Random sampling from the real target data ( where target = 1) 
    real_ones_samples = records_with_ones[np.random.randint(0, len(indexes_of_ones), batch_size)]

    # Generate fake fraud samples using the generator
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    fake_ones_samples = generator.predict(noise)

    # Create labels for real and fake fraud samples
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    # Train the discriminator on real and fake fraud samples
    d_loss_real = discriminator.train_on_batch(real_ones_samples, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_ones_samples, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train generator (freeze discriminator)
    discriminator.trainable = False
    generator.trainable = True

    # Generate synthetic fraud samples and create labels for training the generator
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))

    # Train the generator to generate samples that "fool" the discriminator
    g_loss = gan.train_on_batch(noise, valid_labels)
    g_loss = [i.item() for i in g_loss]
    # Print the progress
    if epoch % 1 == 0:
        print("Epoch: {} - D Loss: {} - G Loss: {}".format(epoch,d_loss,g_loss))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Epoch: 0 - D Loss: [0.7496842  0.50131965 0.29815328] - G Loss: [0.7496673464775085, 0.7496673464775085, 0.5008787512779236, 0.2978910505771637]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Epoch: 1 - D Loss: [0.749365   0.501315   0.29798445] - G Loss: [0.7492984533309937, 0.7492984533309937, 0.5008756518363953, 0.29772329330444336]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Epoch: 2 - D Loss: [0.7494275  0.50131047 0.2978168 ] - G Loss: [0.7494401931762695, 0.7494401931762695, 0.5008726119995117, 0.29755672812461853]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Epoch: 3 - D Loss: [0.74951327 0.5013059  0.2972152 ] - G Loss: [0.7495114803314209, 0.7495114803314209, 0.5008695721626282, 0.29695650935173035]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Epoch: 4 - D Loss: [0.7500193 0.5013013 0.2968346] - G Loss: [0.7499794960021973, 0.7499794960021973, 0.5008665323257446, 0.296577125787735]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Epoch: 5 - D Loss: [0.74972284 0.5012968  0.2968887 ]

In [69]:
noise = np.random.normal(0,1,size = (100,int(latent_dim)))

In [70]:
synthetic_data = generator.predict(noise)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step


In [72]:
synthetic_data.shape

(100, 14)

In [75]:
pd.DataFrame(inverse_scale_data(synthetic_data,scaler),columns = features)

,customer_age,dependent_count,months_on_book,total_relationship_count,months_inactive_12_mon,contacts_count_12_mon,credit_limit,total_revolving_bal,avg_open_to_buy,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio
0,49.630863,3.080832,41.161160,4.759431,2.916995,2.985534,11411.318359,1534.047974,11084.583984,0.878906,5806.187500,77.696602,0.833562,0.350311
1,49.427849,3.044484,40.866386,4.732732,2.917459,2.974849,11578.666992,1533.535400,11419.485352,0.880827,5922.156250,77.812218,0.833808,0.361035
2,49.620205,3.003658,41.134510,4.776831,2.919800,3.021343,11825.421875,1552.955322,11371.003906,0.892208,5764.377441,78.439964,0.818645,0.347553
3,50.339977,2.930320,41.128422,4.640864,2.910113,3.063572,13030.048828,1591.686035,11816.966797,0.885492,5677.535156,77.890305,0.804707,0.363450
4,50.249527,2.929391,41.053093,4.602085,2.922823,3.042693,12996.541016,1595.761719,11993.082031,0.874112,5805.371582,76.912636,0.805229,0.379652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,49.303017,2.995013,42.381237,4.813211,3.048095,2.918695,11023.661133,1457.613525,10865.768555,0.862400,5411.801270,78.952301,0.802648,0.332380
96,50.129490,2.989905,40.635414,4.727664,2.871826,3.039865,12605.060547,1562.877441,11536.345703,0.891549,5794.744629,78.274872,0.820156,0.363032
97,49.805061,2.939523,40.882080,4.729547,2.880733,3.103799,12767.844727,1587.812012,11738.684570,0.903886,5788.503418,79.434303,0.810623,0.367031
98,49.951023,3.036306,40.479431,4.715777,2.871674,2.993943,12212.618164,1537.057495,11434.994141,0.884678,5867.729492,78.019394,0.834372,0.364967
